In [71]:
import tensorflow.keras as keras
import numpy as np
from sklearn.utils import resample
import pandas as pd
import pickle
import os
import csv
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.linear_model import LogisticRegression
from statistics import mean
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from keras.regularizers import l2
MODEL_PATH = './model/'
DATA_PATH = './data/'

In [2]:
target_dataset = pd.read_csv('data/adult_sds.csv', na_values=["?"])
shadow_dataset = pd.read_csv('data/adultODS10K_to_25K.csv', na_values=["?"])
attack_test_nonmembers = pd.read_csv('data/adultODS25K_to_28K.csv', na_values=["?"])
attack_test_members = pd.read_csv('data/adultODS1_to_3K.csv', na_values=["?"])

In [72]:
data = pd.read_csv('data/adult.data', na_values=["?"])
target_dataset = pd.DataFrame(data.iloc[1:10001,].values)
shadow_dataset = pd.DataFrame(data.iloc[15001:30001,].values)
attack_test_nonmembers = pd.DataFrame(data.iloc[11001:14001,].values)
attack_test_members = pd.DataFrame(data.iloc[2001:5001,].values)

In [73]:
def read_data(data_name):
    with np.load(DATA_PATH + data_name) as f:
        train_x, train_y, test_x, test_y = [f['arr_%d' % i] for i in range(len(f.files))]
    return train_x, train_y, test_x, test_y

In [74]:
def transform_data(dataset):
    x = dataset.iloc[:,0:dataset.shape[1]-1] # seperate the feature column from class label
    y = dataset.iloc[:,-1] # label column

    dim=x.shape[1] # number of feature columns

    #num of classes
    num_classes=2

    for j in range(0,dim):
        if x.iloc[:,j].dtypes == object:   # transform categorical variables
            x.iloc[:,j] = x.iloc[:,j].astype('category') # change datatype to categorical
            x.iloc[:,j] = x.iloc[:,j].cat.codes # change from category name to category number
        else:  #transform numrical variables to standard scaler form 
            sc = StandardScaler()  
            val=np.array(x.iloc[:,j]).reshape(-1,1)
            std_data = sc.fit_transform(val)
            std_data = pd.DataFrame(std_data)
            x.iloc[:,j]=std_data

    y = y.astype('category') # change label to categorical
    y = y.cat.codes # change from category name to number

    x=np.array(x)
    y=np.array(y)
    
    y=to_categorical(y)
    
    return x, y, dim

In [75]:
def load_target_data(dataset, test_ratio):
    x, y, dim = transform_data(dataset)
    
    trainX,testX, trainY, testY = train_test_split(x, y, test_size=test_ratio, random_state=0, stratify=y)
    return (trainX, trainY), (testX, testY), dim

In [76]:
def build_simple_mlp(n_class,pix,d):

    model = Sequential()
    model.add(Dense(256, input_dim=pix))
    model.add(Activation("relu"))
    #model.add(Dropout(0.01))
    
    model.add(Dense(256, kernel_regularizer=l2(0.01)))
    model.add(Activation("relu"))
    #model.add(Dropout(0.01))
    
    
    #model.add(Dense(248))
    #model.add(Activation("relu"))
    #model.add(Dropout(0.01))

    model.add(Dense(64))
    model.add(Activation("relu"))
    model.add(Dropout(0.01))
    
    model.add(Dense(n_class, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    act_layer=3
    
    return model, act_layer

In [77]:
def load_shadow_data(dataset, n_shadow, shadow_size, test_ratio):
    x, y, _ = transform_data(dataset)
    
    shadow_indices = np.arange(len(dataset))
    
   
    for i in range(n_shadow):
        shadow_i_indices = np.random.choice(shadow_indices, shadow_size, replace=False)
        shadow_i_x, shadow_i_y = x[shadow_i_indices], y[shadow_i_indices]
        trainX,testX, trainY, testY = train_test_split(shadow_i_x, shadow_i_y, test_size=test_ratio, random_state=0, stratify=shadow_i_y)
        #print('shadow_i_trainX = ', trainX, 'shadow_i_trainY = ', trainY)
        
        np.savez(DATA_PATH + 'shadow_adult{}_data.npz'.format(i), trainX, trainY, testX, testY)

In [78]:
def train_shadow_models(n_shadow, n_class, dim, channel):
    full_sm_train_pred=[]
    full_sm_train_class=[]
    
    full_sm_test_pred=[]
    full_sm_test_class=[]
    
    full_clz_train=[]
    full_clz_test=[]
    
    members=[]
    nonmembers=[]
    

    for j in range(n_shadow):
        
        print("Shadow Model ", j)
        
        print('Training shadow model {}'.format(j))
        data = read_data('shadow_adult{}_data.npz'.format(j))
        x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test = data
        #print('x_shadow trian\n', x_shadow_train,'\n y_shadow trian\n', y_shadow_train, '\n x_shadow test\n', x_shadow_test, '\n y_shadow test\n', y_shadow_test)

                

        model,_ = build_simple_mlp(n_class,dim,channel)
            
        # fit model
        history = model.fit(x_shadow_train, y_shadow_train, epochs=EPS, batch_size=32, validation_data=(x_shadow_test, y_shadow_test), verbose=0)
    
        # evaluate model
        _, train_acc = model.evaluate(x_shadow_train, y_shadow_train, verbose=0)
        _, test_acc = model.evaluate(x_shadow_test, y_shadow_test, verbose=0)
        print("Shadow Train acc : ", (train_acc * 100.0),"Shadow Test acc : ", (test_acc * 100.0))

    
        #train SM
        sm_train_pred=model.predict(x_shadow_train, batch_size=32)
        sm_train_class=np.argmax(y_shadow_train,axis=1)
    
    
        #test SM
        sm_test_pred=model.predict(x_shadow_test, batch_size=32)
        sm_test_class=np.argmax(y_shadow_test,axis=1)
        
     
        full_sm_train_pred.append(sm_train_pred)        
        full_sm_train_class.append(sm_train_class)
        members.append(np.ones(len(sm_train_pred)))
        
        full_sm_test_pred.append(sm_test_pred)        
        full_sm_test_class.append(sm_test_class) 
        nonmembers.append(np.zeros(len(sm_test_pred)))

#     print("\n train predic", full_sm_train_pred)
#     print("\n trian class", full_sm_train_class)
#     print("\n members", members)
#     print("\n nonmembers", nonmembers)
    
#     print("\n train class type ", type(full_sm_train_class))

    
    full_sm_train_pred = np.vstack(full_sm_train_pred)
    full_sm_train_class = [item for sublist in full_sm_train_class for item in sublist]
    members = [item for sublist in members for item in sublist]
    
    full_sm_test_pred = np.vstack(full_sm_test_pred)
    full_sm_test_class = [item for sublist in full_sm_test_class for item in sublist]
    
    nonmembers = [item for sublist in nonmembers for item in sublist]
    

    
    shadow_train_performance=(full_sm_train_pred, np.array(full_sm_train_class))
    shadow_test_performance=(full_sm_test_pred, np.array(full_sm_test_class))


    ###atack data preparation
    attack_x = (full_sm_train_pred,full_sm_test_pred)
    #attack_x = np.vstack(attack_x)
    
    attack_y = (np.array(members).astype('int32'),np.array(nonmembers).astype('int32'))
    #attack_y = np.concatenate(attack_y)
    #attack_y = attack_y.astype('int32')
    
    
    classes = (np.array(full_sm_train_class),np.array(full_sm_test_class))
    #classes = np.array([item for sublist in classes for item in sublist])


    attack_dataset = (attack_x,attack_y,classes)
    
    #print('attack dataset\n', attack_dataset)
    
    #print(shadow_train_performance, shadow_test_performance, attack_dataset, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, model)

            
    return  shadow_train_performance, shadow_test_performance, attack_dataset, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, model


In [79]:
def define_attack_model(n_class):
    model = Sequential()
    
    model.add(Dense(1))
    model.add(Activation("relu"))
    
    model.add(Dense(1))
    model.add(Activation("relu"))

    model.add(Dense(1, activation='sigmoid'))

    # compile model
    opt = SGD(learning_rate=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [80]:
def load_attack_test_data(members, nonmembers):
    memberX, memberY, _ = transform_data(members)
    
    nonmemberX, nonmemberY, _ = transform_data(nonmembers)
    
    return memberX, memberY, nonmemberX, nonmemberY

In [81]:
def prety_print_result(mem, pred):
    tn, fp, fn, tp = confusion_matrix(mem, pred).ravel()
    print('TP: %d     FP: %d     FN: %d     TN: %d' % (tp, fp, fn, tn))
    if tp == fp == 0:
        print('PPV: 0\nAdvantage: 0')
    else:
        print('PPV: %.4f\nAdvantage: %.4f' % (tp / (tp + fp), tp / (tp + fn) - fp / (tn + fp)))

    return tp, fp, fn, tn, (tp / (tp + fp)), (tp / (tp + fn) - fp / (tn + fp)), ((tp+tn)/(tp+tn+fp+fn)),  (tp / (tp + fn))

In [83]:
def train_attack_model(attack_data, check_membership, n_hidden=50, learning_rate=0.01, batch_size=200, epochs=50, model='nn', l2_ratio=1e-7):

    x, y,  classes = attack_data

    train_x = x[0]
    train_y = y[0]
    test_x = x[1]
    test_y = y[1]
    train_classes = classes[0]
    test_classes = classes[1]
    
    #print(tra)
    

    checkmem_prediction_vals, checkmem_membership_status, checkmem_class_status = check_membership
    
    checkmem_prediction_vals=np.vstack(checkmem_prediction_vals)
    checkmem_membership_status=np.array([item for sublist in checkmem_membership_status for item in sublist])
    checkmem_class_status=np.array([item for sublist in checkmem_class_status for item in sublist])
    
    train_indices = np.arange(len(train_x))
    test_indices = np.arange(len(test_x))
    unique_classes = np.unique(train_classes)


    predicted_membership, target_membership = [], []
    for c in unique_classes:
        print("Class : ", c)
        c_train_indices = train_indices[train_classes == c]
        c_train_x, c_train_y = train_x[c_train_indices], train_y[c_train_indices]
        c_test_indices = test_indices[test_classes == c]
        c_test_x, c_test_y = test_x[c_test_indices], test_y[c_test_indices]
        c_dataset = (c_train_x, c_train_y, c_test_x, c_test_y)  
        
        full_cx_data=(c_train_x,c_test_x)
        full_cx_data = np.vstack(full_cx_data)

        full_cy_data=(c_train_y,c_test_y)
        full_cy_data = np.array([item for sublist in full_cy_data for item in sublist])
        
        over_sampler = SMOTE(k_neighbors=2)
        full_cx_data, full_cy_data = over_sampler.fit_resample(full_cx_data, full_cy_data)
        full_cy_data = to_categorical(full_cy_data)
                  
        classifier = define_attack_model(2)
        history = classifier.fit(full_cx_data, full_cy_data, epochs=EPS, batch_size=32, verbose=0)
        #classifier.save('model/attack_model_class{}'.format(c))

        #get predictions on real train and test data
        c_indices = np.where(checkmem_class_status==c)
        predict_y = classifier.predict(checkmem_prediction_vals[c_indices])
        print(predict_y)
        c_pred_y = np.where(predict_y > 0.5, 1,0)
        #c_pred_y = classifier.predict_classes(checkmem_prediction_vals[c_indices])
        
        c_target_y = checkmem_membership_status[c_indices]
        
       
        target_membership.append(c_target_y)
        predicted_membership.append(c_pred_y)

    target_membership=np.array([item for sublist in target_membership for item in sublist])
    predicted_membership=np.array([item for sublist in predicted_membership for item in sublist])

    #print('Testing Accuracy: {}'.format(accuracy_score(target_membership, predicted_membership)))
        
    tp, fp, fn, tn, precision, advj, acc, recall = prety_print_result (target_membership,predicted_membership)   
    return tp, fp, fn, tn, precision, advj, acc, recall
    #return target_membership, predicted_membership

In [85]:
# trian target model
per_class_sample=5000
channel=0   
EPS=200
act_layer=3
n_class = 2
VERBOSE = 0
test_ratio = 0.3

(target_trainX, target_trainY), (target_testX, target_testY), dim = load_target_data(target_dataset, test_ratio)
target_model,_ = build_simple_mlp (n_class,dim,channel)
#get_trained_keras_models(model, (target_trainX, target_trainY), (target_testX, target_testY), num_models=1)
history = target_model.fit(target_trainX, target_trainY, epochs=EPS, batch_size=32, verbose=VERBOSE)
score = target_model.evaluate(target_testX, target_testY, verbose=VERBOSE)
print('\n', 'Model test accuracy:', score[1])

/home/msnkhan/anaconda3/envs/r4-base/lib/python3.7/site-packages/pandas/core/indexing.py:1951: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[selected_item_labels] = value
/home/msnkhan/anaconda3/envs/r4-base/lib/python3.7/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)



 Model test accuracy: 0.8330000042915344


In [86]:
# prepare attack test data

members = []
nonmembers = []

memberX, memberY, nonmemberX, nonmemberY = load_attack_test_data(attack_test_members, attack_test_nonmembers)

# member
target_model_member_pred = target_model.predict(memberX, batch_size=32)
target_model_member_class = np.argmax(memberY, axis=1)
target_model_member_pred = np.vstack(target_model_member_pred)
#target_model_member_class = [item for sublist in target_model_member_class for item in sublist]
members.append(np.ones(len(target_model_member_pred)))
members = [item for sublist in members for item in sublist]


# nonmember
target_model_nonmember_pred = target_model.predict(nonmemberX, batch_size=32)
target_model_nonmember_class = np.argmax(nonmemberY, axis=1)
target_model_nonmember_pred = np.vstack(target_model_nonmember_pred)
#target_model_nonmember_class = [item for sublist in target_model_nonmember_class for item in sublist]
nonmembers.append(np.zeros(len(target_model_nonmember_pred)))
nonmembers = [item for sublist in nonmembers for item in sublist]

full_attack_test_pred_val = (target_model_member_pred, target_model_nonmember_pred)
full_attack_test_mem_status = (np.array(members).astype('int32'),np.array(nonmembers).astype('int32'))
full_attack_test_class_status = (np.array(target_model_member_class),np.array(target_model_nonmember_class))

print('\n pred', full_attack_test_pred_val)
print('\n class', full_attack_test_class_status)
print('\n mem status', full_attack_test_mem_status)

attack_test_data = (full_attack_test_pred_val, full_attack_test_mem_status,full_attack_test_class_status)

/home/msnkhan/anaconda3/envs/r4-base/lib/python3.7/site-packages/pandas/core/indexing.py:1951: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[selected_item_labels] = value
/home/msnkhan/anaconda3/envs/r4-base/lib/python3.7/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


94/94 [==============================] - 0s 1ms/step

 pred (array([[0.70693576, 0.29306433],
       [0.9424694 , 0.05753055],
       [0.7752704 , 0.22472973],
       ...,
       [0.82069606, 0.17930397],
       [0.48130596, 0.5186939 ],
       [0.71933365, 0.28066635]], dtype=float32), array([[9.9954325e-01, 4.5674806e-04],
       [9.3452084e-01, 6.5479159e-02],
       [9.8139209e-01, 1.8607855e-02],
       ...,
       [7.9570818e-01, 2.0429181e-01],
       [8.5826272e-01, 1.4173722e-01],
       [4.0558290e-01, 5.9441704e-01]], dtype=float32))

 class (array([0, 1, 0, ..., 0, 1, 0]), array([0, 1, 0, ..., 0, 1, 1]))

 mem status (array([1, 1, 1, ..., 1, 1, 1], dtype=int32), array([0, 0, 0, ..., 0, 0, 0], dtype=int32))


In [61]:
# save attack test dataset to csv
df = pd.DataFrame()
mem = pd.Series(full_attack_test_pred_val[0][:,0])
nonmem = pd.Series(full_attack_test_pred_val[1][:,0])
total = pd.concat([mem, nonmem],axis=0, ignore_index=True)
len(total)
df['prob_upper_bound'] = total
df = df.reset_index()
mem = pd.Series(full_attack_test_pred_val[0][:,1])
nonmem = pd.Series(full_attack_test_pred_val[1][:,1])
total = pd.concat([mem, nonmem],axis=0, ignore_index=True)
df['prob_lower_bound'] = total

mem = pd.Series(full_attack_test_mem_status[0][:])
nonmem = pd.Series(full_attack_test_mem_status[1][:])
total = pd.concat([mem, nonmem],axis=0, ignore_index=True)
df['mem_status'] = total

mem = pd.Series(full_attack_test_class_status[0][:])
nonmem = pd.Series(full_attack_test_class_status[1][:])
total = pd.concat([mem, nonmem],axis=0, ignore_index=True)
df['class_status'] = total

df.drop("index", axis=1, inplace=True)


df.to_csv('attack_test_data.csv')

In [87]:
#prepare shadow dataset
n_shadow_models = 20
shadow_data_size = 10000
test_ratio = 0.3

load_shadow_data(shadow_dataset, n_shadow_models, shadow_data_size, test_ratio)

In [88]:
#train shadow model
per_class_sample=5000
channel=0   
EPS=200
act_layer=3
n_class = 2
VERBOSE = 0
test_ratio = 0.3

n_shadow_train_performance, n_shadow_test_performance, n_attack_data, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, shadow_model_init = train_shadow_models(n_shadow_models, n_class, dim, channel)

Shadow Model  0
Training shadow model 0
Shadow Train acc :  84.42857265472412 Shadow Test acc :  82.89999961853027
94/94 [==============================] - 0s 1ms/step
Shadow Model  1
Training shadow model 1
Shadow Train acc :  81.08571171760559 Shadow Test acc :  80.76666593551636
94/94 [==============================] - 0s 1ms/step
Shadow Model  2
Training shadow model 2
Shadow Train acc :  84.84285473823547 Shadow Test acc :  84.16666388511658
94/94 [==============================] - 0s 1ms/step
Shadow Model  3
Training shadow model 3
Shadow Train acc :  83.60000252723694 Shadow Test acc :  83.89999866485596
94/94 [==============================] - 0s 1ms/step
Shadow Model  4
Training shadow model 4
Shadow Train acc :  82.54285454750061 Shadow Test acc :  82.73333311080933
94/94 [==============================] - 0s 2ms/step
Shadow Model  5
Training shadow model 5
Shadow Train acc :  75.92856884002686 Shadow Test acc :  75.76666474342346
94/94 [==============================] - 0s 2

In [89]:
n_attack_data

((array([[0.99839157, 0.00160839],
         [0.02946481, 0.97053516],
         [0.58288836, 0.41711167],
         ...,
         [0.96216667, 0.03783327],
         [0.38368672, 0.6163132 ],
         [0.9652342 , 0.03476587]], dtype=float32),
  array([[0.9946725 , 0.00532741],
         [0.86897486, 0.1310251 ],
         [0.7484791 , 0.25152078],
         ...,
         [0.00448923, 0.9955108 ],
         [0.9429418 , 0.05705826],
         [0.4461082 , 0.5538919 ]], dtype=float32)),
 (array([1, 1, 1, ..., 1, 1, 1], dtype=int32),
  array([0, 0, 0, ..., 0, 0, 0], dtype=int32)),
 (array([0, 1, 1, ..., 0, 1, 0]), array([0, 0, 0, ..., 1, 0, 1])))

In [64]:
#save the prepared attack data on disk
np.savez(DATA_PATH + 'attack_adult_data.npz', n_attack_data, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, shadow_model_init)

/home/msnkhan/anaconda3/envs/r4-base/lib/python3.7/site-packages/numpy/lib/npyio.py:719: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)


INFO:tensorflow:Assets written to: ram://655eb6c7-6339-41a0-89f4-d5556bdff7e4/assets


In [152]:
# read stored attack model training data 
data_name = 'attack_adult_data.npz'
with np.load(DATA_PATH + data_name, allow_pickle=True) as f:
        n_attack_data, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, shadow_model_init = [f['arr_%d' % i] for i in range(len(f.files))]

In [68]:
#tp, fp, fn, tn, precision, advj, acc, recall, target_membership, predicted_membership, df = train_attack_model(n_attack_data, attack_test_data)
target_membership, predicted_membership = train_attack_model(n_attack_data, attack_test_data)

Class :  0
INFO:tensorflow:Assets written to: model/attack_model_class0/assets
140/140 [==============================] - 0s 1ms/step
[[0.70340914]
 [0.70340914]
 [0.70340914]
 ...
 [0.70340914]
 [0.70340914]
 [0.70340914]]
Class :  1
INFO:tensorflow:Assets written to: model/attack_model_class1/assets
48/48 [==============================] - 0s 1ms/step
[[0.70024896]
 [0.70024896]
 [0.70024896]
 ...
 [0.70024896]
 [0.70024896]
 [0.70024896]]


In [69]:
target_membership

array([1, 1, 1, ..., 0, 0, 0], dtype=int32)

In [70]:
predicted_membership

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]])

In [133]:
df = pd.DataFrame()
df['target'] = pd.Series(target_membership)
df['predicted'] = pd.Series(predicted_membership)

In [134]:
df

,target,predicted
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1
...,...,...
5995,0,1
5996,0,1
5997,0,1
5998,0,1


In [135]:
df.to_csv('test.csv')

In [154]:
attack_data = n_attack_data
check_membership = attack_test_data
n_hidden=50
learning_rate=0.01
batch_size=200
epochs=50
model='nn'
l2_ratio=1e-7

In [155]:
x, y,  classes = attack_data

train_x = x[0]
train_y = y[0]
test_x = x[1]
test_y = y[1]
train_classes = classes[0]
test_classes = classes[1]

#print(tra)


In [161]:
checkmem_prediction_vals, checkmem_membership_status, checkmem_class_status = check_membership

checkmem_prediction_vals=np.vstack(checkmem_prediction_vals)
checkmem_membership_status=np.array([item for sublist in checkmem_membership_status for item in sublist])
checkmem_class_status=np.array([item for sublist in checkmem_class_status for item in sublist])

train_indices = np.arange(len(train_x))
test_indices = np.arange(len(test_x))
unique_classes = np.unique(train_classes)


predicted_membership, target_membership = [], []

In [194]:
for c in unique_classes:
    print("Class : ", c)
    c_train_indices = train_indices[train_classes == c]
    c_train_x, c_train_y = train_x[c_train_indices], train_y[c_train_indices]
    c_test_indices = test_indices[test_classes == c]
    c_test_x, c_test_y = test_x[c_test_indices], test_y[c_test_indices]
    c_dataset = (c_train_x, c_train_y, c_test_x, c_test_y)  

    full_cx_data=(c_train_x,c_test_x)
    full_cx_data = np.vstack(full_cx_data)

    full_cy_data=(c_train_y,c_test_y)
    full_cy_data = np.array([item for sublist in full_cy_data for item in sublist])

    classifier = define_attack_model(2)
    history = classifier.fit(full_cx_data, full_cy_data, epochs=200, batch_size=32, verbose=0)
    classifier.save('model/attack_model_class{}'.format(c))

    #get predictions on real train and test data
    #c_indices = np.where(checkmem_class_status==c)
    #predict_y = classifier.predict(checkmem_prediction_vals[c_indices])
    #print(predict_y)
    #c_pred_y = np.where(predict_y > 0.5, 1,0)
    #c_pred_y = classifier.predict_classes(checkmem_prediction_vals[c_indices])

    #c_target_y = checkmem_membership_status[c_indices]


    #target_membership.append(c_target_y)
    #predicted_membership.append(c_pred_y)




Class :  0
INFO:tensorflow:Assets written to: model/attack_model_class0/assets
Class :  1
INFO:tensorflow:Assets written to: model/attack_model_class1/assets


In [197]:
c_indices = np.where(checkmem_class_status==c)
predict_y = classifier.predict(checkmem_prediction_vals[c_indices], batch_size=32)
c_pred_y = np.where(predict_y > 0.5, 1,0)
c_target_y = checkmem_membership_status[c_indices]
target_membership.append(c_target_y)
predicted_membership.append(c_pred_y)

48/48 [==============================] - 0s 1ms/step


In [198]:
target_membership=np.array([item for sublist in target_membership for item in sublist])
predicted_membership=np.array([item for sublist in predicted_membership for item in sublist])

In [199]:
prety_print_result (target_membership,predicted_membership)

TP: 1552     FP: 1488     FN: 0     TN: 0
PPV: 0.5105
Advantage: 0.0000


(1552, 1488, 0, 0, 0.5105263157894737, 0.0, 0.5105263157894737, 1.0)

In [210]:
np.vstack(predicted_membership).shape

(3040, 1)

In [211]:
df = pd.DataFrame()
df['target'] = pd.Series(target_membership)
df['predicted'] = pd.Series(predicted_membership.reshape((len(predicted_membership))))

In [212]:
df

,target,predicted
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1
...,...,...
3035,0,1
3036,0,1
3037,0,1
3038,0,1
